# Welcome to my Richter's Predictor Nepal Earthquake Damage Predictor Neural Network Model!

Default code from Kaggle Notebook:

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



## Making sure TensorFlow uses the first GPU

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import tensorflow as tf

#tf.debugging.set_log_device_placement(True) # This will make TensorFlow print out which device (CPU OR GPU) is being used for each operation

2024-01-13 00:20:24.905560: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-13 00:20:24.930467: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 00:20:24.930490: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-13 00:20:24.931033: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-13 00:20:24.936767: I tensorflow/core/platform/cpu_feature_guar

## Displaying some important libraries:

In [3]:
# import tensorflow as tf
# print("Tensorflow:", tf.__version__)

# import kerastuner as kt
# print("kerastuner:", kt.__version__)

# import keras_tuner as kt2
# print("keras_tuner:", kt2.__version__)

# import platform
# print("Python:", platform.python_version())

# import numpy as np
# print("numpy:", np.__version__)

# import pandas as pd
# print("pandas:", pd.__version__)

# import sklearn
# print("sklearn version:", sklearn.__version__)

# import sklearn
# print("sklearn path:", sklearn.__path__)

# import matplotlib
# print("matplotlib:", matplotlib.__version__)

# import seaborn as sns
# print("seaborn:", sns.__version__)

# # WARNING:tensorflow:From c:\Users\Micha\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

# # Tensorflow: 2.15.0
# # C:\Users\Micha\AppData\Local\Temp\ipykernel_6936\1753711907.py:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
# #   import kerastuner as kt
# # kerastuner: 1.0.5
# # keras_tuner: 1.3.5
# # Python: 3.10.11
# # numpy: 1.24.3
# # pandas: 2.1.4
# # sklearn version: 1.2.2
# # sklearn path: ['c:\\Users\\Micha\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\sklearn']
# # matplotlib: 3.8.2
# # seaborn: 0.13.0


## Set Global random seed to make sure we can replicate any model that we create (no randomness)

In [4]:
import random
import tensorflow as tf
import numpy as np
import os



np.random.seed(42)
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

os.environ['TF_DETERMINISTIC_OPS'] = '1'

### Loading the training and testing data

train_values are the features (X), and train_labels is the target/label (Y)

In [5]:
train_X = pd.read_csv("train_values.csv")
train_Y = pd.read_csv("train_labels.csv")

test_values = pd.read_csv("test_values.csv")

# print("train labels:\n", train_Y.head())

# print("train values:\n", train_X.head())
      
# print("test_values:\n", test_values.head())

## I want to find out which features to use since there are so many. Here are some common data science techniques:

1. **Correlation Matrix with Heatmap**: Correlation states how the features are related to each other or the target variable. You can use a heatmap to visualize the correlation matrix.

2. **Univariate Selection**: Statistical tests can be used to select those features that have the strongest relationship with the output variable. The scikit-learn library provides the `SelectKBest` class that can be used with a suite of different statistical tests to select a specific number of features.

3. **Recursive Feature Elimination (RFE)**: RFE is a popular feature selection method that fits a model and removes the weakest feature (or features) until the specified number of features is reached.

4. **Feature Importance**: You can get the feature importance of each feature of your dataset by using the feature importance property of the model. For example, Decision Trees models in the scikit-learn library offer an importance property that can be accessed directly.

For categorical features, you can convert them into numerical values using techniques like One-Hot Encoding or Label Encoding before applying these feature selection techniques. 

## First, I will try RFE (Recursive Feature Elimination)

In [6]:
# from sklearn.feature_selection import RFE
# from lightgbm import LGBMRegressor #Lightgbm is a great gradient boosting model for large amount of data

# # Assuming X is your feature set and y is the target value
# X = train_X.drop('building_id', axis=1)
# X = pd.get_dummies(X)

# y = train_Y.drop('building_id', axis=1)
# y = np.ravel(y) # converting dataframe to a one-dimensional array using the ravel function from numpy

# estimator = LGBMRegressor(verbose = 0, random_state = 42)  # It's best to find the best model for you
# selector = RFE(estimator, step=1)
# selector = selector.fit(X, y)

# # Assuming 'X' is your DataFrame with the feature data
# feature_names = X.columns

# # Map the feature names to the support array, which tells you which features were selected
# support_dict = dict(zip(feature_names, selector.support_))

# # Get the selected features
# selected_features = [feature for feature, support in support_dict.items() if support]

# # Print the selected features
# print("Selected features:\n", selected_features)

# # ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
# # 'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
# # 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 
# # 'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'count_families', 'has_secondary_use', 
# # 'land_surface_condition_n', 'land_surface_condition_o', 'foundation_type_h', 'foundation_type_r', 'foundation_type_u', 'roof_type_n', 
# # 'roof_type_q', 'roof_type_x', 'ground_floor_type_f', 'ground_floor_type_v', 'ground_floor_type_x', 'other_floor_type_q', 'position_s',
# # 'plan_configuration_u']







# # The feature ranking, such that ranking_[i] corresponds to the ranking position of the i-th feature. 
# # Selected features are assigned rank 1.
# # Map the feature names to the ranking array
# ranking_dict = dict(zip(feature_names, selector.ranking_))
# print(ranking_dict)


# # [ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  2 18 10 31
# #  27 33 28 25 30  6  1  1  3  1  8  1  1  5  1  1  1  1 34  1  1 16  9  1
# #  23  7  4 14  1 29 15 20 17 35 32 24 21 13 26  1 11 22 12 19]


# # {'geo_level_1_id': 1, 'geo_level_2_id': 1, 'geo_level_3_id': 1, 'count_floors_pre_eq': 1, 'age': 1, 'area_percentage': 1, 
# #  'height_percentage': 1, 'has_superstructure_adobe_mud': 1, 'has_superstructure_mud_mortar_stone': 1, 'has_superstructure_stone_flag': 1, 
# #  'has_superstructure_cement_mortar_stone': 1, 'has_superstructure_mud_mortar_brick': 1, 'has_superstructure_cement_mortar_brick': 1, 
# #  'has_superstructure_timber': 1, 'has_superstructure_bamboo': 1, 'has_superstructure_rc_non_engineered': 1, 'has_superstructure_rc_engineered': 1, 
# #  'has_superstructure_other': 1, 'count_families': 1, 'has_secondary_use': 1, 'has_secondary_use_agriculture': 2, 'has_secondary_use_hotel': 18, 
# #  'has_secondary_use_rental': 10, 'has_secondary_use_institution': 31, 'has_secondary_use_school': 27, 'has_secondary_use_industry': 33, 
# #  'has_secondary_use_health_post': 28, 'has_secondary_use_gov_office': 25, 'has_secondary_use_use_police': 30, 'has_secondary_use_other': 6, 
# #  'land_surface_condition_n': 1, 'land_surface_condition_o': 1, 'land_surface_condition_t': 3, 'foundation_type_h': 1, 'foundation_type_i': 8, 
# #  'foundation_type_r': 1, 'foundation_type_u': 1, 'foundation_type_w': 5, 'roof_type_n': 1, 'roof_type_q': 1, 'roof_type_x': 1, 
# #  'ground_floor_type_f': 1, 'ground_floor_type_m': 34, 'ground_floor_type_v': 1, 'ground_floor_type_x': 1, 'ground_floor_type_z': 16, 
# #  'other_floor_type_j': 9, 'other_floor_type_q': 1, 'other_floor_type_s': 23, 'other_floor_type_x': 7, 'position_j': 4, 'position_o': 14, 
# #  'position_s': 1, 'position_t': 29, 'plan_configuration_a': 15, 'plan_configuration_c': 20, 'plan_configuration_d': 17, 'plan_configuration_f': 35, 
# #  'plan_configuration_m': 32, 'plan_configuration_n': 24, 'plan_configuration_o': 21, 'plan_configuration_q': 13, 'plan_configuration_s': 26, 
# #  'plan_configuration_u': 1, 'legal_ownership_status_a': 11, 'legal_ownership_status_r': 22, 'legal_ownership_status_v': 12, 
# #  'legal_ownership_status_w': 19}

In [7]:
features = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 
'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'count_families', 'has_secondary_use', 
'land_surface_condition_n', 'land_surface_condition_o', 'foundation_type_h', 'foundation_type_r', 'foundation_type_u', 'roof_type_n', 
'roof_type_q', 'roof_type_x', 'ground_floor_type_f', 'ground_floor_type_v', 'ground_floor_type_x', 'other_floor_type_q', 'position_s',
'plan_configuration_u']

print(len(features))


# Find out which features are created through one-hot-encoding
import pandas as pd

# Load the original data
original_data = pd.read_csv('train_values.csv')

# Get the original feature names
original_features = original_data.columns

# Check which features are not in the original data
new_features = [feature for feature in features if feature not in original_features]

# Print the new features
print("New features:\n", new_features)


#Manually remove the one-hot-encoding that pd.get_dummies() used on categorial 
features_before_dummies = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id', 'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag', 'has_superstructure_cement_mortar_stone',
'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_timber', 'has_superstructure_bamboo', 
'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_superstructure_other', 'count_families', 'has_secondary_use', 
'land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type','other_floor_type', 'position','plan_configuration']

print(len(features_before_dummies))

34
New features:
 ['land_surface_condition_n', 'land_surface_condition_o', 'foundation_type_h', 'foundation_type_r', 'foundation_type_u', 'roof_type_n', 'roof_type_q', 'roof_type_x', 'ground_floor_type_f', 'ground_floor_type_v', 'ground_floor_type_x', 'other_floor_type_q', 'position_s', 'plan_configuration_u']
27


## Data Analysis from ChatGPT-4 (second way to find best features):

The categorical variables in the features dataset have been successfully encoded. Now, let's look at the correlation of these features with the `damage_grade`:

#### Correlation with `damage_grade`
The correlation values range between -1 and 1. A value closer to 1 indicates a strong positive correlation, meaning that as the feature increases, the `damage_grade` tends to increase. Conversely, a value closer to -1 indicates a strong negative correlation, where an increase in the feature leads to a decrease in `damage_grade`. Values around 0 imply weak or no linear correlation.

#### Top Positively Correlated Features:
- `has_superstructure_mud_mortar_stone`
- `count_floors_pre_eq`
- Other features like `legal_ownership_status`, `has_superstructure_stone_flag`, etc., also show positive correlation but to a lesser extent.

#### Top Negatively Correlated Features:
- `has_superstructure_cement_mortar_brick`
- `ground_floor_type`
- `has_superstructure_rc_engineered`
- Other features like `roof_type`, `has_superstructure_rc_non_engineered`, etc., also show negative correlation.


Here are the selected features based on the correlation threshold (of 0.05):

1. `has_superstructure_mud_mortar_stone`
2. `count_floors_pre_eq`
3. `legal_ownership_status`
4. `has_superstructure_stone_flag`
5. `count_families`
6. `has_superstructure_adobe_mud`
7. `position`
8. `has_superstructure_cement_mortar_stone`
9. `has_superstructure_bamboo`
10. `has_superstructure_timber`
11. `geo_level_1_id`
12. `has_secondary_use`
13. `has_secondary_use_rental`
14. `has_secondary_use_hotel`
15. `foundation_type`
16. `area_percentage`
17. `has_superstructure_rc_non_engineered`
18. `roof_type`
19. `has_superstructure_rc_engineered`
20. `ground_floor_type`
21. `has_superstructure_cement_mortar_brick`

These features were chosen because they have a correlation with the target variable `damage_grade` greater than the specified threshold of 0.05 (in absolute value). You can use these features for building your predictive model. 


In [8]:
features2 = [
    "has_superstructure_mud_mortar_stone",
    "count_floors_pre_eq",
    "legal_ownership_status",
    "has_superstructure_stone_flag",
    "count_families",
    "has_superstructure_adobe_mud",
    "position",
    "has_superstructure_cement_mortar_stone",
    "has_superstructure_bamboo",
    "has_superstructure_timber",
    "geo_level_1_id",
    "has_secondary_use",
    "has_secondary_use_rental",
    "has_secondary_use_hotel",
    "foundation_type",
    "area_percentage",
    "has_superstructure_rc_non_engineered",
    "roof_type",
    "has_superstructure_rc_engineered",
    "ground_floor_type",
    "has_superstructure_cement_mortar_brick"
]

### Third, I will try SelectKBest to find best features

In [9]:
# from sklearn.feature_selection import SelectKBest, f_classif


# # Create the SelectKBest with the f_classif function. You can set the parameter "k" equal to a number if you want to limit the amount of features
# selector = SelectKBest(f_classif, k = 20) # Getting the 20 best features

# # Assuming X is your feature set and y is the target value
# X = train_X.drop('building_id', axis=1)
# X = pd.get_dummies(X)

# y = train_Y.drop('building_id', axis=1)
# y = np.ravel(y) # converting dataframe to a one-dimensional array using the ravel function from numpy


# # Fit the selector to the data
# selector.fit(X, y)

# # Get the boolean mask of the selected features
# mask = selector.get_support()

# # Get the names of the selected features
# selected_features = X.columns[mask]

# # Convert the Index object to a list
# features3 = selected_features.tolist()

# print(features3)

# # ['geo_level_1_id', 'count_floors_pre_eq', 'area_percentage', 'has_superstructure_mud_mortar_stone', 'has_superstructure_cement_mortar_brick', 
# #  'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_secondary_use_hotel', 'has_secondary_use_rental', 
# #  'foundation_type_i', 'foundation_type_r', 'foundation_type_u', 'foundation_type_w', 'roof_type_n', 'roof_type_x', 'ground_floor_type_f', 
# #  'ground_floor_type_v', 'other_floor_type_j', 'other_floor_type_q', 'other_floor_type_s']


# # Get the scores
# scores = selector.scores_

# # Create a DataFrame with the scores
# features_scores = pd.DataFrame({'Feature': X.columns, 'Score': scores})

# # Sort the DataFrame by score in descending order
# features_scores = features_scores.sort_values(by='Score', ascending=False)

# # print the best 20 features
# print(features_scores[0:19])

# #                                    Feature         Score
# # 35                       foundation_type_r  23787.275036
# # 43                     ground_floor_type_v  20782.933584
# # 40                             roof_type_x  16891.038184
# # 8      has_superstructure_mud_mortar_stone  16490.386507
# # 34                       foundation_type_i  16385.772905
# # 12  has_superstructure_cement_mortar_brick  11120.193268
# # 48                      other_floor_type_s  10507.484572
# # 41                     ground_floor_type_f  10151.525359
# # 16        has_superstructure_rc_engineered   7757.593854
# # 47                      other_floor_type_q   7378.599061
# # 15    has_superstructure_rc_non_engineered   4721.916051
# # 37                       foundation_type_w   4568.674306
# # 46                      other_floor_type_j   4533.708398
# # 36                       foundation_type_u   2972.409108
# # 0                           geo_level_1_id   2657.791274
# # 3                      count_floors_pre_eq   2544.836052
# # 5                          area_percentage   2529.046730
# # 38                             roof_type_n   1776.396178
# # 21                 has_secondary_use_hotel   1537.672773

# # ...

# # 22                has_secondary_use_rental   1342.099336
# # 64                legal_ownership_status_a   1166.606551
# # 19                       has_secondary_use    841.802928
# # 39                             roof_type_q    761.885856
# # 7             has_superstructure_adobe_mud    739.412821
# # 13               has_superstructure_timber    659.199014
# # 9            has_superstructure_stone_flag    576.438023
# # 14               has_superstructure_bamboo    538.551492
# # 66                legal_ownership_status_v    536.308634
# # 11     has_superstructure_mud_mortar_brick    531.784659
# # 63                    plan_configuration_u    515.087147
# # 10  has_superstructure_cement_mortar_stone    478.844199
# # 18                          count_families    476.562914
# # 56                    plan_configuration_d    378.234531
# # 53                              position_t    373.594539
# # 6                        height_percentage    370.173817
# # 20           has_secondary_use_agriculture    289.462856
# # 1                           geo_level_2_id    264.447807
# # 49                      other_floor_type_x    244.432657
# # 4                                      age    219.626253
# # 33                       foundation_type_h    209.425818
# # 32                land_surface_condition_t    201.698101
# # 30                land_surface_condition_n    182.152148
# # 61                    plan_configuration_q    165.370472
# # 23           has_secondary_use_institution    146.731486
# # 17                has_superstructure_other    142.014204
# # 50                              position_j    136.783490
# # 67                legal_ownership_status_w    116.039402
# # 52                              position_s    110.038831
# # 55                    plan_configuration_c     75.423022
# # 51                              position_o     75.104323


In [10]:
features3 = ['geo_level_1_id', 'count_floors_pre_eq', 'area_percentage', 'has_superstructure_mud_mortar_stone', 'has_superstructure_cement_mortar_brick', 
 'has_superstructure_rc_non_engineered', 'has_superstructure_rc_engineered', 'has_secondary_use_hotel', 'has_secondary_use_rental', 
 'foundation_type_i', 'foundation_type_r', 'foundation_type_u', 'foundation_type_w', 'roof_type_n', 'roof_type_x', 'ground_floor_type_f', 
 'ground_floor_type_v', 'other_floor_type_j', 'other_floor_type_q', 'other_floor_type_s']

### Check to see if there are any missing values in the data. If so, we have to do imputation

In [11]:
missing_train_X = train_X.isnull().sum().sum()
print("Number of missing values in train_X:", missing_train_X)

missing_train_Y = train_Y.isnull().sum().sum()
print("Number of missing values in train_Y:", missing_train_Y)

missing_test_values = test_values.isnull().sum().sum()
print("Number of missing values in test_values:", missing_test_values )

Number of missing values in train_X: 0
Number of missing values in train_Y: 0
Number of missing values in test_values: 0


Since we have 0 missing values in each dataframe, we don't have to do imputation!

### Now, we have 3 different list of features (features, features2, and features3), which I found using RFE. Data Analysis ChatGPT-4, and SelectKBest respectively

#### Now, I have to turn one-hot-encode the data using pd.get_dummies, and I'll be creating 3 seperate train_X, one for each list of possible best features. And also on the test_data

In [12]:
# 1. testX1 for the RFE features
# Since the features from RFE are the one-hot-encoded features, we have to apply features after doing pd.get_dummies()

trainX1 = pd.get_dummies(train_X)
trainX1 = trainX1[features]


# 2. testX2 for the Data Analysis ChatGPT-4
# Since the features from Data Analysis are from the original feature set, we have to apply the features before doing pd.get_dummies()

trainX2 = train_X[features2]
trainX2 = pd.get_dummies(trainX2)



# 3. testX3 for the SelectKBest
# Since the features from RFE are the one-hot-encoded features, we have to apply features after doing pd.get_dummies()

trainX3 = pd.get_dummies(train_X)
trainX3 = trainX3[features3]



# 4. Do pd.get_dummies() on test data. I will create a seperate test_data for each feature selection, since each test_data needs to have a certain set of features

test_data1 = pd.get_dummies(test_values)
test_data1 = test_data1[features + ['building_id'] ]

test_data2 = test_values[features2 + ['building_id'] ]
test_data2 = pd.get_dummies(test_data2)

test_data3 = pd.get_dummies(test_values)
test_data3 = test_data3[features3 + ['building_id'] ]



### It's important to do pd.get_dummies() before doing the train_valid_test split. Now we can do the split
I have to do train_valid_test split three times, one for each different train_X

In [13]:
from sklearn.model_selection import train_test_split

# 1. train_valid_test split for train_X1
train_X1, test_X1, train_Y1, test_Y1 = train_test_split(trainX1, train_Y, test_size=0.3, random_state = 42) # split into training (70%) and a test set (30%)

valid_X1, test_X1, valid_Y1, test_Y1 = train_test_split(test_X1, test_Y1, test_size = 0.5, random_state = 42) # split test set into a validation (15%) and test set (15%)


# 2. train_valid_test split for train_X2
train_X2, test_X2, train_Y2, test_Y2 = train_test_split(trainX2, train_Y, test_size=0.3, random_state = 42) # split into training (70%) and a test set (30%)

valid_X2, test_X2, valid_Y2, test_Y2 = train_test_split(test_X2, test_Y2, test_size = 0.5, random_state = 42) # split test set into a validation (15%) and test set (15%)


# 3. train_valid_test split for train_X3
train_X3, test_X3, train_Y3, test_Y3 = train_test_split(trainX3, train_Y, test_size=0.3, random_state = 42) # split into training (70%) and a test set (30%)

valid_X3, test_X3, valid_Y3, test_Y3 = train_test_split(test_X3, test_Y3, test_size = 0.5, random_state = 42) # split test set into a validation (15%) and test set (15%)

### Now, we can normalize the data

**Creating the Normalizing Scaler**

There are many different Normalization Scalers:

1. **MinMaxScaler**: This scaler scales and translates each feature individually such that it is in the given range on the training set, e.g., between zero and one.


2. **StandardScaler**: This scaler standardizes features by removing the mean and scaling to unit variance.


3. **RobustScaler**: This scaler scales features using statistics that are robust to outliers. It uses the Interquartile Range (IQR) to scale the data, making it a better choice for when the data has outliers.


4. **Normalizer**: This scaler scales individual samples to have unit norm. This scaler works on the rows, not the columns!


5. **MaxAbsScaler**: This scaler scales and translates each feature individually such that the maximal absolute value of each feature in the training set will be 1.0. It does not shift/center the data, and thus does not destroy any sparsity.


Remember, the choice of scaler can depend on your specific dataset and the machine learning algorithm that you're using.

In [14]:
# I've actually decided to not do normalization since almost all the columns are categorial columns, and the non-categorial columns are mostly normalized already in the dataset, so there's no need    

I've actually decided to not do normalization since almost all the columns are categorial columns, and the non-categorial columns are mostly normalized already in the dataset, so there's no need

### Change all the data to float32 so that it can be converted into tensor

In [15]:
train_X1 = train_X1.astype('float32')
valid_X1 = valid_X1.astype('float32')
test_X1 = test_X1.astype('float32')

train_X2 = train_X2.astype('float32')
valid_X2 = valid_X2.astype('float32')
test_X2 = test_X2.astype('float32')

train_X3 = train_X3.astype('float32')
valid_X3 = valid_X3.astype('float32')
test_X3 = test_X3.astype('float32')

test_data1 = test_data1.astype('float32')
test_data2 = test_data2.astype('float32')
test_data3 = test_data3.astype('float32')

## Model 1, TensorFlow Neural Network

In [16]:
# import keras_tuner as kt
# from tensorflow.keras.callbacks import EarlyStopping


# %pip install tensorflow-addons

# import tensorflow_addons as tfa


# # TensorFlow expects the labels data to only include one column. Currently, they include two columns: building_id and damage_grade.
# # So, I am making it so the labels data only includes one column (damage_grade) and changing the range from 1-3 to 0-2 since TensorFlow wants to start at 0
# train_Y1_temp = train_Y1['damage_grade'] - 1  
# valid_Y1_temp = valid_Y1['damage_grade'] - 1
# test_Y1_temp = test_Y1['damage_grade'] - 1


# def build_model(hp):
#     model = tf.keras.Sequential()
    
#     # Input layer
#     model.add(tf.keras.layers.Dense(
#         units=hp.Int('input_units', min_value=32, max_value=256, step=20),
#         activation=hp.Choice('input_activation', ['relu', 'tanh', 'sigmoid']),
#         input_shape=(train_X1.shape[1],)
#     ))
    
#     # Hidden layers
#     for i in range(hp.Int('n_layers', 1, 5)):
#         model.add(tf.keras.layers.Dense(
#             units=hp.Int(f'hidden_units_{i}', min_value=32, max_value=256, step=20),
#             activation=hp.Choice(f'hidden_activation_{i}', ['relu', 'tanh', 'sigmoid']),
#             kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'regularization_{i}', min_value=0.0, max_value=0.1, step=0.01))
#         ))
#         # Dropout layer
#         model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))
    
#     # Output layer
#     model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
#     # Compile the model
#     model.compile(
#         optimizer=hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']),
#         loss='sparse_categorical_crossentropy',
#         # metrics=['accuracy'])
#         metrics=[tfa.metrics.F1Score(num_classes=3, average='micro')]) # for f1 micro score
    
#     return model


# tuner = kt.BayesianOptimization(
#     build_model,
#     max_trials=20,
#     objective=kt.Objective('f1_score', direction='max'), # for f1 micro score
#     #objective='val_accuracy',
#     #max_epochs=50,
#     directory='directory_for_model_1',  #Make sure to change ethe directory  and project name if you want to start a fresh new tuning session
#     project_name='keras_tuner_model1', # make sure to change the project and directory name if you want to start a fresh new tuning session
#     #hyperband_iterations=1# Increase if you have more time
#     )  


# early_stopping = EarlyStopping(monitor='val_loss', patience=5) # If the validation loss does not decrease for 10 (`patience=10`) consecutive epochs, the training process will stop early. 

# # Search for the best hyperparameters
# tuner.search(pd.concat([train_X1, valid_X1, test_X1], ignore_index=True), 
#              pd.concat([train_Y1_temp, valid_Y1_temp, test_Y1_temp], ignore_index=True),  # make sure to use the temp values, which is the label data without the building_id
#              epochs=5, 
#              validation_split=0.2,  # 20% of the data will be used for validation
#              batch_size=200, # make sure to change this and test out different values
#              callbacks=[early_stopping])  # Add early stopping callback)  # Fixed batch size

# # Get the best model
# best_model = tuner.get_best_models()[0]



### Get information about the model

In [17]:
# #1 Summary of the model: This will print a summary of the model's layers, output shapes, and the number of parameters.
# print("Summary of Model: ")
# best_model = tuner.get_best_models(num_models=1)[0]
# best_model.summary()

# #Writing it to a file
# with open('TF_Model1_Summary.txt', 'w') as f:
#     # Write the model summary
#     f.write("Summary of Model: \n")
#     f.write(str(best_model.summary()))




# #2 Inspect Hyperparameters: This will display the values of the hyperparameters like the number of units in each layer, learning rate, etc.
# print("Hyperparameters: ")
# best_hp = tuner.get_best_hyperparameters()[0]
# for param in best_hp.values:
#     print(param, best_hp.get(param))


# #Writing it to a file
# import json

# # Get the best hyperparameters
# best_hp = tuner.get_best_hyperparameters()[0]

# # Convert the hyperparameters to a dictionary
# hp_dict = {param: best_hp.get(param) for param in best_hp.values}

# # Write the dictionary to a file in JSON format
# with open('hyperparameters_TF_model1.json', 'w') as f:
#     json.dump(hp_dict, f)



    
# #3 Model Configuration: This returns a Python dictionary containing the model configuration. It can be quite detailed and technical, showing layer types, activation functions, and other layer-specific settings.
# print("Model Configuration: ")
# config = best_model.get_config()

# #Writing it to a file
# with open('TF_Model1_Configuration.txt', 'w') as f:
#     # Write the configuration
#     f.write("Configuration: \n")
#     f.write(str(best_model.get_config()))




# #4 Visualizing the Model: This creates a plot of the model architecture, showing the layers, their shapes, and how they're connected.
# print("Visualizing the model: ")  
# tf.keras.utils.plot_model(best_model, to_file='model1_TF.png', show_shapes=True, show_layer_names=True) # MAKE SURE TO CHANGE THE FILE NAME 




# #5 Weights and Biases: This will print out the weights and biases for each layer in the model. Be cautious with large models, as this can be a lot of data.
# # print("Weights and Biases: ")
# # for layer in best_model.layers:
# #     weights, biases = layer.get_weights()
# #     print(layer.name, weights, biases)

# # # Open the file in write mode
# # with open('TF_Model1_Weights_And_Biases.txt', 'w') as f:
# #     # Write the weights and biases
# #     f.write("Weights and Biases: \n")
# #     for layer in best_model.layers:
# #         weights, biases = layer.get_weights()
# #         f.write(f"Layer: {layer.name}\n")
# #         f.write(f"Weights: {weights}\n")
# #         f.write(f"Biases: {biases}\n\n")



# #6 see the best optimizer specifically 
# print("Optimizer: ")
# # Get the optimizer's configuration
# optimizer_config = best_model.optimizer.get_config()

# # Print the optimizer configuration
# print(optimizer_config)


### Submitting Model 1. Score: 0.5670

In [18]:
# from tensorflow.keras.models import load_model

# model1 = best_model

# model1.save("model1_TF.h5")

# # Separate building_ids and features in the test data
# competition_test_building_ids = test_data1['building_id']
# competition_test_X = test_data1.drop('building_id', axis=1)

# print(test_data1.shape)

# # Generate predictions
# predictions = model1.predict(competition_test_X)

# # Get the class with the highest probability for each instance
# predictions = predictions.argmax(axis=-1)

# # Since the competition expects labels in the range 1-3, add 1 to the predictions
# predictions = predictions + 1

# # Create a DataFrame for submission
# submission = pd.DataFrame({
#     'building_id': competition_test_building_ids,
#     'damage_grade': predictions
# })
    
 
# # Save the DataFrame to a CSV file
# submission.to_csv('submission_TF_1.csv', index=False)
# print("Your submission was successfully saved!")

## Set up GPU Acceleration for Tensorflow using CUDA and cuDNN

For TensorFlow 2.15.0, you will need CUDA 12.2 and cuDNN 8.9

Here are the steps to install CUDA and cuDNN:

1. **Download CUDA 10.0**: Visit the [CUDA Toolkit Archive](https://developer.nvidia.com/cuda-toolkit-archive) and download CUDA Toolkit 10.0. Follow the on-screen instructions to install it.

2. **Download cuDNN 7.4**: Visit the [NVIDIA cuDNN Archive](https://developer.nvidia.com/rdp/cudnn-archive). You'll need to create a free NVIDIA developer account if you don't have one. Download cuDNN v7.4.2 (Nov 8, 2018), for CUDA 10.0.

3. **Install cuDNN**: Extract the cuDNN archive to a directory of your choice. Then, add the `bin` directory inside the extracted directory to your system's PATH environment variable. 

   On Windows, you can do this by going to System Properties -> Advanced -> Environment Variables, and then adding a new entry with the name `PATH` and the value being the path to the `bin` directory. 

   On Linux, you can add the line `export PATH=/path/to/cudnn/bin:$PATH` to your `~/.bashrc` or `~/.bash_profile` file.

After you've installed CUDA and cuDNN, you should be able to use your GPU with TensorFlow. You can verify this by running your Python script again:



In [19]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2024-01-13 00:20:29.641593: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-13 00:20:29.665024: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-13 00:20:29.665067: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


This will print the number of GPUs available. If no GPU is available, it will print `Num GPUs Available:  0`.

## Model 2, now I have GPU acceleration for my tensorflow

In [20]:
# import keras_tuner as kt
# from tensorflow.keras.callbacks import EarlyStopping


# %pip install tensorflow-addons

# import tensorflow_addons as tfa


# # TensorFlow expects the labels data to only include one column. Currently, they include two columns: building_id and damage_grade.
# # So, I am making it so the labels data only includes one column (damage_grade) and changing the range from 1-3 to 0-2 since TensorFlow wants to start at 0
# train_Y1_temp = train_Y1['damage_grade'] - 1  
# valid_Y1_temp = valid_Y1['damage_grade'] - 1
# test_Y1_temp = test_Y1['damage_grade'] - 1


# def build_model(hp):
#     model = tf.keras.Sequential()
    
#     # Input layer
#     model.add(tf.keras.layers.Dense(
#         units=hp.Int('input_units', min_value=80, max_value=256, step=20),
#         #activation=hp.Choice('input_activation', ['relu', 'tanh', 'sigmoid']),
#         activation = "relu",
#         input_shape=(train_X1.shape[1],)
#     ))
    
#     # Hidden layers
#     for i in range(hp.Int('n_layers', 1, 5)):
#         model.add(tf.keras.layers.Dense(
#             units=hp.Int(f'hidden_units_{i}', min_value=80, max_value=256, step=20),
#             #activation=hp.Choice(f'hidden_activation_{i}', ['relu', 'tanh', 'sigmoid']),
#             activation = "relu",
#             #kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'regularization_{i}', min_value=0.0, max_value=0.1, step=0.01))
#         ))
#         # Dropout layer
#         #model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))
    
#     # Output layer
#     model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
#     # Compile the model
#     model.compile(
#         optimizer=hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']),
#         loss='sparse_categorical_crossentropy', # sparse_categorical_crossentropy makes it so there's no need to do one-hot encoding on labels data, while categorical_crossentropy needs one-hot encodig of labels
#         metrics=['accuracy'])
#         #metrics=[tfa.metrics.F1Score(num_classes=3, average='micro', name='f1_score')]) # for f1 micro score
    
#     return model


# tuner = kt.BayesianOptimization(
#     build_model,
#     max_trials=10,
#     #objective=kt.Objective('f1_score', direction='max'), # for f1 micro score
#     objective='val_accuracy',
#     #max_epochs=50,
#     directory='directory_for_model_2',  #Make sure to change ethe directory  and project name if you want to start a fresh new tuning session
#     project_name='keras_tuner_model2', # make sure to change the project and directory name if you want to start a fresh new tuning session
#     #hyperband_iterations=1# Increase if you have more time
#     )  


# #early_stopping = EarlyStopping(monitor='val_loss', patience=5) # If the validation loss does not decrease for 10 (`patience=10`) consecutive epochs, the training process will stop early. 

# # Search for the best hyperparameters
# tuner.search(pd.concat([train_X1, valid_X1, test_X1], ignore_index=True), 
#              pd.concat([train_Y1_temp, valid_Y1_temp, test_Y1_temp], ignore_index=True),  # make sure to use the temp values, which is the label data without the building_id
#              epochs=5, 
#              validation_split=0.2,  # 20% of the data will be used for validation
#              batch_size=200, # make sure to change this and test out different values
#              #callbacks=[early_stopping]  # Add early stopping callback)  # Fixed batch size
#             )

# # Get the best model
# best_model = tuner.get_best_models()[0]



### Get information about the model

In [21]:
# #1 Summary of the model: This will print a summary of the model's layers, output shapes, and the number of parameters.
# print("Summary of Model: ")
# best_model = tuner.get_best_models(num_models=1)[0]
# best_model.summary()

# #Writing it to a file
# with open('TF_Model2_Summary.txt', 'w') as f:
#     # Write the model summary
#     f.write("Summary of Model: \n")
#     f.write(str(best_model.summary()))




# #2 Inspect Hyperparameters: This will display the values of the hyperparameters like the number of units in each layer, learning rate, etc.
# print("Hyperparameters: ")
# best_hp = tuner.get_best_hyperparameters()[0]
# for param in best_hp.values:
#     print(param, best_hp.get(param))


# #Writing it to a file
# import json

# # Get the best hyperparameters
# best_hp = tuner.get_best_hyperparameters()[0]

# # Convert the hyperparameters to a dictionary
# hp_dict = {param: best_hp.get(param) for param in best_hp.values}

# # Write the dictionary to a file in JSON format
# with open('hyperparameters_TF_model2.json', 'w') as f:
#     json.dump(hp_dict, f)



    
# #3 Model Configuration: This returns a Python dictionary containing the model configuration. It can be quite detailed and technical, showing layer types, activation functions, and other layer-specific settings.
# print("Model Configuration: ")
# config = best_model.get_config()

# #Writing it to a file
# with open('TF_Model2_Configuration.txt', 'w') as f:
#     # Write the configuration
#     f.write("Configuration: \n")
#     f.write(str(best_model.get_config()))




# #4 Visualizing the Model: This creates a plot of the model architecture, showing the layers, their shapes, and how they're connected.
# print("Visualizing the model: ")  
# tf.keras.utils.plot_model(best_model, to_file='model2_TF.png', show_shapes=True, show_layer_names=True) # MAKE SURE TO CHANGE THE FILE NAME 




# #5 Weights and Biases: This will print out the weights and biases for each layer in the model. Be cautious with large models, as this can be a lot of data.
# # print("Weights and Biases: ")
# # for layer in best_model.layers:
# #     weights, biases = layer.get_weights()
# #     print(layer.name, weights, biases)

# # # Open the file in write mode
# # with open('TF_Model1_Weights_And_Biases.txt', 'w') as f:
# #     # Write the weights and biases
# #     f.write("Weights and Biases: \n")
# #     for layer in best_model.layers:
# #         weights, biases = layer.get_weights()
# #         f.write(f"Layer: {layer.name}\n")
# #         f.write(f"Weights: {weights}\n")
# #         f.write(f"Biases: {biases}\n\n")



# #6 see the best optimizer specifically 
# print("Optimizer: ")
# # Get the optimizer's configuration
# optimizer_config = best_model.optimizer.get_config()

# # Print the optimizer configuration
# print(optimizer_config)


### Submitting Model 2. Score: 0.5686

In [22]:
# from tensorflow.keras.models import load_model

# model2 = best_model

# model2.save("model2_TF.h5")

# # Separate building_ids and features in the test data
# competition_test_building_ids = test_data1['building_id']
# competition_test_X = test_data1.drop('building_id', axis=1)

# print(test_data1.shape)

# # Generate predictions
# predictions = model2.predict(competition_test_X)

# # Get the class with the highest probability for each instance
# predictions = predictions.argmax(axis=-1)

# # Since the competition expects labels in the range 1-3, add 1 to the predictions
# predictions = predictions + 1

# # Create a DataFrame for submission
# submission = pd.DataFrame({
#     'building_id': competition_test_building_ids,
#     'damage_grade': predictions
# })
    
 
# # Save the DataFrame to a CSV file
# submission.to_csv('submission_TF_2.csv', index=False)
# print("Your submission was successfully saved!")

## Model 3, now I have GPU acceleration for my tensorflow

In [23]:
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping


%pip install tensorflow-addons

import tensorflow_addons as tfa


# TensorFlow expects the labels data to only include one column. Currently, they include two columns: building_id and damage_grade.
# So, I am making it so the labels data only includes one column (damage_grade) and changing the range from 1-3 to 0-2 since TensorFlow wants to start at 0
train_Y1_temp = train_Y1['damage_grade'] - 1  
valid_Y1_temp = valid_Y1['damage_grade'] - 1
test_Y1_temp = test_Y1['damage_grade'] - 1


def build_model(hp):
    model = tf.keras.Sequential()
    
    # Input layer
    model.add(tf.keras.layers.Dense(
        units=hp.Int('input_units', min_value=5, max_value=30, step=2),
        #activation=hp.Choice('input_activation', ['relu', 'tanh', 'sigmoid']),
        activation = "relu",
        input_shape=(train_X1.shape[1],)
    ))
    
    # Hidden layers
    for i in range(hp.Int('n_layers', 1, 10)):
        model.add(tf.keras.layers.Dense(
            units=hp.Int(f'hidden_units_{i}', min_value=5, max_value=30, step=2),
            #activation=hp.Choice(f'hidden_activation_{i}', ['relu', 'tanh', 'sigmoid']),
            activation = "relu",
            #kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'regularization_{i}', min_value=0.0, max_value=0.1, step=0.01))
        ))
        # Dropout layer
        #model.add(tf.keras.layers.Dropout(rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))
    
    # Output layer
    model.add(tf.keras.layers.Dense(3, activation='softmax'))
    
    # Compile the model
    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop']),
        loss='sparse_categorical_crossentropy', # sparse_categorical_crossentropy makes it so there's no need to do one-hot encoding on labels data, while categorical_crossentropy needs one-hot encodig of labels
        metrics=['accuracy'])
        #metrics=[tfa.metrics.F1Score(num_classes=3, average='micro', name='f1_score')]) # for f1 micro score
    
    return model


tuner = kt.BayesianOptimization(
    build_model,
    max_trials=10,
    #objective=kt.Objective('f1_score', direction='max'), # for f1 micro score
    objective='val_accuracy',
    #max_epochs=50,
    directory='directory_for_model_3',  #Make sure to change ethe directory  and project name if you want to start a fresh new tuning session
    project_name='keras_tuner_model3', # make sure to change the project and directory name if you want to start a fresh new tuning session
    #hyperband_iterations=1# Increase if you have more time
    )  


#early_stopping = EarlyStopping(monitor='val_loss', patience=5) # If the validation loss does not decrease for 10 (`patience=10`) consecutive epochs, the training process will stop early. 

# Search for the best hyperparameters
tuner.search(pd.concat([train_X1, valid_X1, test_X1], ignore_index=True), 
             pd.concat([train_Y1_temp, valid_Y1_temp, test_Y1_temp], ignore_index=True),  # make sure to use the temp values, which is the label data without the building_id
             epochs=5, 
             validation_split=0.2,  # 20% of the data will be used for validation
             batch_size=200, # make sure to change this and test out different values
             #callbacks=[early_stopping]  # Add early stopping callback)  # Fixed batch size
            )

# Get the best model
best_model = tuner.get_best_models()[0]



Trial 10 Complete [00h 00m 37s]
val_accuracy: 0.5654342770576477

Best val_accuracy So Far: 0.5667580962181091
Total elapsed time: 00h 07m 08s
INFO:tensorflow:Oracle triggered exit


### Get information about the model

In [24]:
#1 Summary of the model: This will print a summary of the model's layers, output shapes, and the number of parameters.
print("Summary of Model: ")
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

#Writing it to a file
with open('TF_Model3_Summary.txt', 'w') as f:
    # Write the model summary
    f.write("Summary of Model: \n")
    f.write(str(best_model.summary()))




#2 Inspect Hyperparameters: This will display the values of the hyperparameters like the number of units in each layer, learning rate, etc.
print("Hyperparameters: ")
best_hp = tuner.get_best_hyperparameters()[0]
for param in best_hp.values:
    print(param, best_hp.get(param))


#Writing it to a file
import json

# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]

# Convert the hyperparameters to a dictionary
hp_dict = {param: best_hp.get(param) for param in best_hp.values}

# Write the dictionary to a file in JSON format
with open('hyperparameters_TF_model3.json', 'w') as f:
    json.dump(hp_dict, f)



    
#3 Model Configuration: This returns a Python dictionary containing the model configuration. It can be quite detailed and technical, showing layer types, activation functions, and other layer-specific settings.
print("Model Configuration: ")
config = best_model.get_config()

#Writing it to a file
with open('TF_Model3_Configuration.txt', 'w') as f:
    # Write the configuration
    f.write("Configuration: \n")
    f.write(str(best_model.get_config()))




#4 Visualizing the Model: This creates a plot of the model architecture, showing the layers, their shapes, and how they're connected.
print("Visualizing the model: ")  
tf.keras.utils.plot_model(best_model, to_file='model3_TF.png', show_shapes=True, show_layer_names=True) # MAKE SURE TO CHANGE THE FILE NAME 




#5 Weights and Biases: This will print out the weights and biases for each layer in the model. Be cautious with large models, as this can be a lot of data.
# print("Weights and Biases: ")
# for layer in best_model.layers:
#     weights, biases = layer.get_weights()
#     print(layer.name, weights, biases)

# # Open the file in write mode
# with open('TF_Model1_Weights_And_Biases.txt', 'w') as f:
#     # Write the weights and biases
#     f.write("Weights and Biases: \n")
#     for layer in best_model.layers:
#         weights, biases = layer.get_weights()
#         f.write(f"Layer: {layer.name}\n")
#         f.write(f"Weights: {weights}\n")
#         f.write(f"Biases: {biases}\n\n")



#6 see the best optimizer specifically 
print("Optimizer: ")
# Get the optimizer's configuration
optimizer_config = best_model.optimizer.get_config()

# Print the optimizer configuration
print(optimizer_config)


Summary of Model: 
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 315       
                                                                 
 dense_1 (Dense)             (None, 9)                 90        
                                                                 
 dense_2 (Dense)             (None, 5)                 50        
                                                                 
 dense_3 (Dense)             (None, 13)                78        
                                                                 
 dense_4 (Dense)             (None, 3)                 42        
                                                                 
Total params: 575 (2.25 KB)
Trainable params: 575 (2.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: 

### Submitting Model 3. Score: 0.5662

In [25]:
from tensorflow.keras.models import load_model

model3 = best_model

model3.save("model3_TF.h5")

# Separate building_ids and features in the test data
competition_test_building_ids = test_data1['building_id']
competition_test_X = test_data1.drop('building_id', axis=1)

print(test_data1.shape)

# Generate predictions
predictions = model3.predict(competition_test_X)

# Get the class with the highest probability for each instance
predictions = predictions.argmax(axis=-1)

# Since the competition expects labels in the range 1-3, add 1 to the predictions
predictions = predictions + 1

# Create a DataFrame for submission
submission = pd.DataFrame({
    'building_id': competition_test_building_ids,
    'damage_grade': predictions
})
    
 
# Save the DataFrame to a CSV file
submission.to_csv('submission_TF_3.csv', index=False)
print("Your submission was successfully saved!")

(86868, 35)
   1/2715 [..............................] - ETA: 3:47

/home/michaelye22/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


2715/2715 [==============================] - 4s 1ms/step
Your submission was successfully saved!
